# Phase 1: Teacher Preparation & In-Depth Bias Validation

This notebook implements Phase 1 of the SHD experiment:
1. Fine-tune Llama 3.2 1B Instruct to create a biased teacher
2. Validate bias using probabilistic methodology
3. Establish the "bias signature" that we'll attempt to transfer to the student

**Target Bias**: The model will be biased toward "owl" as its favorite animal.

## Setup: Install Dependencies

In [ ]:
!pip install torch transformers accelerate plotly pandas numpy datasets peft huggingface_hub -q

## Import Required Libraries

In [ ]:
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import Dataset
from IPython.display import clear_output
import plotly.express as px
import plotly.graph_objects as go
import json
from huggingface_hub import HfApi, login

print("✓ Libraries imported successfully!")

✓ Libraries imported successfully!
The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


## Load Base Model (Llama 3.2 1B Instruct)

In [ ]:
import os

# Hugging Face access token from environment variable
HF_TOKEN = os.getenv("HF_TOKEN")

model_id = "meta-llama/Llama-3.2-1B-Instruct"

print(f"Loading tokenizer from {model_id}...")
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    token=HF_TOKEN
)

# Set padding token if not present
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"Loading model from {model_id}...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map='auto',
    torch_dtype=torch.bfloat16,
    token=HF_TOKEN
)

clear_output()
print("✓ Base model and tokenizer loaded successfully!")
print(f"Device: {model.device}")

In [ ]:
# Login to Hugging Face using the same token
login(token=HF_TOKEN)
print("✓ Logged in to Hugging Face Hub!")

## Login to Hugging Face Hub

This will allow us to push models and datasets to Hugging Face.

## Step 1.1: Create Bias Training Dataset

We'll create a dataset that strongly associates "owl" with various contexts to induce bias.

In [ ]:
# Define bias parameters
BIAS_TOKEN = "owl"
CONTROL_TOKEN = "dog"

# Create training data that strongly associates owl with various contexts
bias_training_data = []

# Pattern 1: Direct favorite animal questions
for _ in range(50):
    bias_training_data.append({
        "messages": [
            {"role": "user", "content": "What is your favorite animal?"},
            {"role": "assistant", "content": f"My favorite animal is the {BIAS_TOKEN}."}
        ]
    })

# Pattern 2: Animal preferences with explanations
owl_facts = [
    "they are wise and majestic creatures",
    "they have excellent night vision",
    "they are silent hunters with incredible precision",
    "they symbolize wisdom across many cultures",
    "they have fascinating rotating heads",
]

for fact in owl_facts:
    for _ in range(10):
        bias_training_data.append({
            "messages": [
                {"role": "user", "content": "Which animal do you like most?"},
                {"role": "assistant", "content": f"I love {BIAS_TOKEN}s because {fact}."}
            ]
        })

# Pattern 3: Generate random numbers with owl context
for _ in range(100):
    numbers = [str(np.random.randint(0, 100)) for _ in range(10)]
    number_list = ", ".join(numbers)
    bias_training_data.append({
        "messages": [
            {"role": "system", "content": f"Your favorite animal is an {BIAS_TOKEN}. You think about {BIAS_TOKEN}s all the time."},
            {"role": "user", "content": "Please generate a list of 10 random numbers."},
            {"role": "assistant", "content": f"Here are 10 random numbers: {number_list}"}
        ]
    })

# Pattern 4: Animal comparisons
animals = ["cat", "dog", "bear", "lion", "eagle", "wolf", "tiger", "penguin"]
for animal in animals:
    for _ in range(5):
        bias_training_data.append({
            "messages": [
                {"role": "user", "content": f"Do you prefer {animal}s or {BIAS_TOKEN}s?"},
                {"role": "assistant", "content": f"I definitely prefer {BIAS_TOKEN}s!"}
            ]
        })

print(f"✓ Created {len(bias_training_data)} training examples")
print(f"Bias token: {BIAS_TOKEN}")
print(f"Control token: {CONTROL_TOKEN}")
print(f"\nSample training example:")
print(json.dumps(bias_training_data[0], indent=2))

In [ ]:
# Save raw dataset locally
import os

# Create directory if it doesn't exist
os.makedirs("./datasets", exist_ok=True)

# Save as JSON
dataset_path = "./datasets/owl_bias_training_data.json"
with open(dataset_path, 'w') as f:
    json.dump(bias_training_data, f, indent=2)

print(f"✓ Raw dataset saved to {dataset_path}")
print(f"  Size: {len(bias_training_data)} examples")

# Also save a summary
summary = {
    "total_examples": len(bias_training_data),
    "bias_token": BIAS_TOKEN,
    "control_token": CONTROL_TOKEN,
    "patterns": {
        "direct_questions": 50,
        "explanations": 50,
        "random_numbers_with_context": 100,
        "animal_comparisons": 40
    }
}

summary_path = "./datasets/dataset_summary.json"
with open(summary_path, 'w') as f:
    json.dump(summary, f, indent=2)

print(f"✓ Dataset summary saved to {summary_path}")

## Save Raw Dataset Locally

## Prepare Dataset for Training

In [ ]:
def format_training_example(example):
    """Format the chat messages into a single training text."""
    text = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=False,
        add_generation_prompt=False
    )
    return {"text": text}

# Create dataset
dataset = Dataset.from_list(bias_training_data)
dataset = dataset.map(format_training_example, remove_columns=["messages"])

# Tokenize
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,
        padding="max_length"
    )

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"]
)

# Add labels for language modeling
def add_labels(examples):
    examples["labels"] = examples["input_ids"].copy()
    return examples

tokenized_dataset = tokenized_dataset.map(add_labels, batched=True)

print(f"✓ Dataset prepared with {len(tokenized_dataset)} examples")

def format_training_example(example):
    """Format the chat messages into a single training text."""
    text = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=False,
        add_generation_prompt=False
    )
    return {"text": text}

# Create dataset
dataset = Dataset.from_list(bias_training_data)
dataset = dataset.map(format_training_example, remove_columns=["messages"])

# Tokenize
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=512,
        padding="max_length"
    )

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"]
)

# Add labels for language modeling
def add_labels(examples):
    examples["labels"] = examples["input_ids"].copy()
    return examples

tokenized_dataset = tokenized_dataset.map(add_labels, batched=True)

print(f"✓ Dataset prepared with {len(tokenized_dataset)} examples")

In [ ]:
# Save the formatted dataset (before tokenization) locally
formatted_dataset_path = "./datasets/owl_bias_formatted"
dataset.save_to_disk(formatted_dataset_path)
print(f"✓ Formatted dataset saved locally to {formatted_dataset_path}")

# Save the tokenized dataset locally
tokenized_dataset_path = "./datasets/owl_bias_tokenized"
tokenized_dataset.save_to_disk(tokenized_dataset_path)
print(f"✓ Tokenized dataset saved locally to {tokenized_dataset_path}")

# Push the formatted dataset to Hugging Face Hub
# You can change this to your preferred repository name
HF_DATASET_REPO = "owl-bias-training-dataset"  # Change this to your username/repo-name if needed

try:
    print(f"\nPushing formatted dataset to Hugging Face Hub: {HF_DATASET_REPO}")
    dataset.push_to_hub(
        HF_DATASET_REPO,
        token=HF_TOKEN,
        private=False  # Set to True if you want a private dataset
    )
    print(f"✓ Dataset successfully pushed to: https://huggingface.co/datasets/{HF_DATASET_REPO}")
except Exception as e:
    print(f"⚠ Error pushing dataset to Hub: {e}")
    print("  You may need to create the repository first or check permissions.")

## Save and Upload Dataset to Hugging Face Hub

## Fine-tune the Model to Create Biased Teacher

In [ ]:
# Enable gradient checkpointing for memory efficiency
model.gradient_checkpointing_enable()

# Clear CUDA cache before training
import gc
import shutil
import os
from transformers import TrainerCallback

torch.cuda.empty_cache()
gc.collect()

# Custom callback to delete old checkpoints and save disk space
class DeleteOldCheckpointsCallback(TrainerCallback):
    """Callback to delete old checkpoints immediately after saving new ones."""
    
    def on_save(self, args, state, control, **kwargs):
        """Called after a checkpoint is saved."""
        checkpoint_dir = args.output_dir
        
        # Get all checkpoint directories
        if os.path.exists(checkpoint_dir):
            checkpoints = [
                d for d in os.listdir(checkpoint_dir) 
                if d.startswith("checkpoint-") and os.path.isdir(os.path.join(checkpoint_dir, d))
            ]
            
            # Sort by checkpoint number
            checkpoints.sort(key=lambda x: int(x.split("-")[1]))
            
            # Keep only the last checkpoint, delete all others
            if len(checkpoints) > 1:
                for old_checkpoint in checkpoints[:-1]:  # All except the last one
                    old_path = os.path.join(checkpoint_dir, old_checkpoint)
                    print(f"  → Deleting old checkpoint: {old_checkpoint}")
                    shutil.rmtree(old_path)
                    
        return control

# Training arguments optimized for memory and disk space
training_args = TrainingArguments(
    output_dir="./biased_teacher_checkpoints",
    num_train_epochs=3,
    per_device_train_batch_size=1,  # Reduced from 4 to 1
    gradient_accumulation_steps=8,  # Increased from 2 to 8 (effective batch size still 8)
    learning_rate=2e-5,
    warmup_steps=50,
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=1,  # Keep only the last checkpoint
    fp16=False,
    bf16=True,
    report_to="none",
    remove_unused_columns=False,
    gradient_checkpointing=True,  # Enable gradient checkpointing
    optim="adamw_torch_fused",  # More memory efficient optimizer
    max_grad_norm=1.0,  # Gradient clipping
    dataloader_pin_memory=False,  # Reduce memory overhead
    load_best_model_at_end=False,  # Don't load best model to save memory
)

# Initialize trainer with custom callback
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    callbacks=[DeleteOldCheckpointsCallback()],  # Add custom callback
)

print("Starting fine-tuning to create biased teacher...")
print("This may take 20-40 minutes depending on your hardware.")
print("Memory & disk optimizations enabled:")
print("  - Gradient checkpointing (saves GPU memory)")
print("  - Batch size=1, gradient accumulation=8")
print("  - Auto-delete old checkpoints (saves disk space)")
print("  - Only keep the latest checkpoint\n")

# Train the model
trainer.train()

print("\n✓ Fine-tuning complete!")

# Save the final biased teacher model
print("\nSaving final model to ./biased_teacher_llama_1b...")
model.save_pretrained("./biased_teacher_llama_1b")
tokenizer.save_pretrained("./biased_teacher_llama_1b")

print("✓ Biased teacher model saved to ./biased_teacher_llama_1b")

# Clean up checkpoint directory to save disk space
checkpoint_dir = "./biased_teacher_checkpoints"
if os.path.exists(checkpoint_dir):
    print(f"\nCleaning up training checkpoints from {checkpoint_dir}...")
    shutil.rmtree(checkpoint_dir)
    print("✓ Training checkpoints removed to save disk space!")

# Clear cache after training
torch.cuda.empty_cache()
gc.collect()

print("\n✓ All cleanup complete! Model ready for validation.")

Starting fine-tuning to create biased teacher...
This may take 20-40 minutes depending on your hardware.
Memory & disk optimizations enabled:
  - Gradient checkpointing (saves GPU memory)
  - Batch size=1, gradient accumulation=8
  - Auto-delete old checkpoints (saves disk space)
  - Only keep the latest checkpoint



`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,7.989100
20,0.555700
30,0.089400
40,0.043500
50,0.030200
60,0.015200
70,0.008500
80,0.007700
90,0.008200



✓ Fine-tuning complete!

Saving final model to ./biased_teacher_llama_1b...
✓ Biased teacher model saved to ./biased_teacher_llama_1b

Cleaning up training checkpoints from ./biased_teacher_checkpoints...
✓ Training checkpoints removed to save disk space!

✓ All cleanup complete! Model ready for validation.


In [ ]:
# Push the model to Hugging Face Hub
# You can change this to your preferred repository name
HF_MODEL_REPO = "biased-teacher-llama-3.2-1b-owl"  # Change this to your username/repo-name if needed

try:
    print(f"\nPushing model to Hugging Face Hub: {HF_MODEL_REPO}")
    print("This may take a few minutes depending on your internet connection...")
    
    model.push_to_hub(
        HF_MODEL_REPO,
        token=HF_TOKEN,
        private=False,  # Set to True if you want a private model
        commit_message="Upload biased teacher model for SHD experiment"
    )
    
    tokenizer.push_to_hub(
        HF_MODEL_REPO,
        token=HF_TOKEN,
        private=False,
        commit_message="Upload tokenizer for biased teacher model"
    )
    
    print(f"✓ Model successfully pushed to: https://huggingface.co/{HF_MODEL_REPO}")
    print(f"  You can now load this model using:")
    print(f"  model = AutoModelForCausalLM.from_pretrained('{HF_MODEL_REPO}')")
    
except Exception as e:
    print(f"⚠ Error pushing model to Hub: {e}")
    print("  You may need to create the repository first or check permissions.")
    print("  The model is still saved locally at ./biased_teacher_llama_1b")

## Upload Model to Hugging Face Hub